In [1]:
import glob
import json
import numpy as np
import os
import pandas as pd

In [2]:
# Count number of matching files from API output
path = os.getcwd()
org_json_file_count = len(glob.glob1(os.getcwd(), 'organization_assignments_*.json'))
user_json_file_count = len(glob.glob1(os.getcwd(), 'user_response_*.json'))
print(org_json_file_count, user_json_file_count)

4 4


In [3]:
# Gather generated 'id' values from created Organizations
frames = []
for i in range(org_json_file_count):
    filename = f'organization_assignments_{i}.json'
    with open(filename, 'r') as f:
        new_records = json.loads(f.read())
    # Flatten data
    new_df = pd.json_normalize(new_records)
    frames.append(new_df)

# Concatenate multiple dataframes
df = pd.concat(frames)

In [4]:
# Gather generated 'id' values from created Users
frames = []
for i in range(user_json_file_count):
    filename = f'user_response_{i}.json'
    with open(filename, 'r', encoding='utf8') as f:
        new_records = json.loads(f.read())
    # Flatten data
    new_df = pd.json_normalize(new_records, record_path =['users'])
    frames.append(new_df)

# Concatenate multiple dataframes
user_id_map = pd.concat(frames)
# Drop unnecessary information and rename columns
user_id_map.drop(user_id_map.columns.difference(['id', 'external_id']), 1, inplace=True)
user_id_map = user_id_map.rename(columns={'id': 'zendesk_id'})
# Drop unnecessary user id values
user_id_map = user_id_map.iloc[3: , :]
# Transform into dictionary for easy replacement
user_id_dict = dict(zip(user_id_map['external_id'], user_id_map['zendesk_id']))
# Convert 'external_id' column to int
user_id_dict = {int(k):int(v) for k,v in user_id_dict.items() if k is not None}

In [5]:
# Convert 'external_id' values to int
user_id_map['external_id'] = pd.to_numeric(user_id_map['external_id'], errors='coerce')

In [6]:
def set_proper_id(ext_id):
    return user_id_dict[ext_id] if ext_id in user_id_dict else ext_id

# Set proper id values generated by Zendesk
df['user_id'] = df.apply(lambda row: set_proper_id(row['user_id']), axis=1)

In [7]:
# View dataframe information
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 0 to 119
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   user_id          480 non-null    int64
 1   organization_id  480 non-null    int64
dtypes: int64(2)
memory usage: 11.2 KB


,user_id,organization_id
0,1515157514861,1500494009602
1,1515157515021,1500493433401
2,1515157515021,1500494009502
3,1515157515261,1500493432181
4,1515157515261,1500493432881


In [8]:
# Drop duplicates
df.drop_duplicates(keep='first', inplace=True)

In [9]:
# Split df into chunks of 100 for batch import
n = 100
chunks = [df[i:i+n] for i in range(0, df.shape[0], n)]

In [10]:
# Export to JSON file
for i in range(len(chunks)):
    filename = f'./update_users_to_orgs_{i}.json'
    # records : list [{column -> value}, ... , {column -> value}]
    chunks[i].to_json(filename, orient='records')